# Part 1: Implementing Logistic Regression from Scratch

In this part of the lab, you will be requested to implement logistic regression from scratch. This means you will need to make use of gradient descent to find the parameters of the model.


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.datasets import make_classification
from sklearn.preprocessing import PolynomialFeatures
from sklearn.model_selection import train_test_split 
from sklearn.metrics import f1_score

In [ ]:
def plot_loss(title, values):
    '''
    This function will allow us to check the evolution of the loss function during gradient descent
    Inputs:
    Title - title of the plot
    Values - values to be plotted
    '''
    plt.figure(figsize=(3, 3))
    plt.plot(values)
    plt.title(title)
    plt.ylabel('loss')
    plt.xlabel('iteration')
    plt.show()

For this exercise, we will make use of the function [make_classification](https://scikit-learn.org/stable/modules/generated/sklearn.datasets.make_classification.html) from scikit-learn, which generates random data for an k-class classification problem. We will use k=2 to stick to a binary classification problem.

In [ ]:
X, y = make_classification(n_features=10, n_redundant=0, n_informative=6, n_classes=2, n_clusters_per_class=1)

## Question 1: make_classification
Check the documentation of the function to determine what is the role of the following parameters:

    1. n_redundant
    2. n_informative
    3. n_repeated

Based on your findings, how many useless features does the dataset contain?

Your answer here:

## Exercise 1: Scaling input features
When different input features have extremely different ranges of values, it is common to rescale them so they have comparable ranges. We standardize input values by centering them to result in a zero mean and a standard deviation of one (this transformation is sometimes called the z-score). That is, if $\mu_j$ is the mean of the values of the j-th feature across the N samples in the input dataset, and $\sigma_j$ is the standard deviation of the values of features j-th across the input dataset, we can replace each feature $x_i^j$ by a new feature $x^{'j}_i$ computed as follows:

$$\mu_j = \frac{1}{N}\sum^{N}_{i=1} x_i^j$$

$$\sigma_j = \sqrt{\frac{1}{N}\sum^{N}_{i=1} x_i^j - \mu_j)^2} $$

$$x^{'j}_i = \dfrac{x_i^j - \mu_j}{\sigma_j}$$

### Task 1.1: Implement feature scaling
Implement below the function standardize, which estimates the mean and standard deviation of each feature in the dataset and then standardizes all the input features.

**Hint:** Check the documentation of the functions mean, std and divide from numpy.

In [ ]:
def standardize(X, mean = None, stdev = None):
    '''
    Transforms the input data using the z-score. 
    If the mean and stdev are provided, the function only performs the transformation.
    Otherwise, it first estimates the mean and standard deviation
    Inputs:
    X- Data to standardize
    mean - vector with means of each feature (default None)
    stdev - vector with standard deviation of each feature (default None)
    Outputs:
    X_stand - Standardized data
    mean - Mean of the data
    stdev - standard deviation of the data
    '''
    #YOUR CODE HERE
    
    return X_stand, mean, stdev

## Exercise 2: Implementing and Running Logistic Regression
### Task 2.1: Implement Logistic Regression
Below you will see the skeleton of the Logistic Regression class. Some of its functions have already been implemented. Have a look at them and try to understand them.

Afterwards, you will need to complete the following:

1. **function sigmoid** - Computes the sigmoid function given an input (*see slide 15 of the Logistic Regression slide deck*)

2. **function loss_function** - Estimates the cross-entropy loss given an input matrix X, a vector of labels y and the weights. Attention: In the course's slides, we estimated the loss by suming over all elements of the training set. For efficiency purposes, estimate it using matrix/vector computations. You may have a look into the linear regression lab for inspiration on how to do this

3. **function gradient_descent_step** - Performs an update of the weights for logistic regression. Using matrix notation this is expressed as:
$$ \mathbf{w}^{(\tau+1)} = \mathbf{w}^{(\tau)} + \dfrac{\alpha}{N}\mathbf{X}^T\left(\mathbf{y}-\sigma\left(\mathbf{X}\mathbf{w}\right)\right)$$

4. **function prediction** - Predicts new labels y_pred given an input matrix (*see slide 24 from Logistic regression slide deck*)


In [ ]:
class LogisticRegression:
    
    def initialize_weights(self,X):
        '''
        Initializes the parameters so that the have the same dimensions as the input data + 1
        Inputs:
        X - input data matrix of dimensions N x D
        
        Outputs:
        weights - model parameters initialized to zero size (D + 1) x 1
        '''
        weights = np.zeros((X.shape[1]+1,1))
        
        return weights
    
    def initialize_X(self,X):
        '''
        Reshapes the input data so that it can handle w_0
        Inputs:
        X - input data matrix of dimensions N x D
        Outputs:
        X - matrix of size N x (D + 1)
        '''
        X = PolynomialFeatures(1).fit_transform(X) #Adds a one to the matrix so it copes with w_0
        
        return X
    
    def sigmoid(self,z):
        '''
        Implements the sigmoid function
        Input:
        z - input variable 
        
        Output:
        1/(1+exp(-z))
        '''
        # YOUR CODE HERE 
        return sig
        
    def loss_function(self,X,y,w):
        '''
        Implements the cross-entropy loss. See Eq 1, Slide 21 from the Logistic Regression slide deck as a reminder.
        Note that the expression in slide 21 is not using a matrix notation. 
        Input:
        X - Input matrix of size N x (D + 1)
        y - Label vector of size N x 1
        w - Parameters vector of size (D + 1) x 1
        
        Output: 
        Estimation of the cross-entropy loss given the input, labels and parameters (scalar value)
        '''
        
        #1) Estimate Xw
        #YOUR CODE HERE
        
        #2) Estimate sigmoid of Xw
         #YOUR CODE HERE
        
        #3) estimate log(sig) and log(1-sig)
         #YOUR CODE HERE
        
        #4) Combine point 3 with the labels and sum over all elements to obtain the final estimate
        loss =  #YOUR CODE HERE
        
        return loss
    
    def gradient_descent_step(self,X, y, w, alpha):
        '''
        Implements a gradient descent step for logistic regression
        Input:
        X - Input matrix of size N x (D + 1)
        y - Label vector of size N x 1
        w - Parameters vector of size (D + 1) x 1
        alpha - Learning rate 
        Output: 
        Updated weights
        '''
        
        w = #YOUR CODE HERE
        
        return w
    
    def fit(self,X,y,alpha=0.01,iter=10, epsilon = 0.0001):
        '''
        Learning procedure of the logistic regression model
        Input:
        X - Input matrix of size N x (D + 1)
        y - Label vector of size N x 1
        alpha - Learning rate (default value 0.01)
        iter - Number of iterations to perform for gradient descent (default 10)
        epsilon - stopping criterion (default 0.0001)
        Output: 
        List of values of the loss function during the gradient descent iterations
        '''
        weights = self.initialize_weights(X) #Initializes the weights of the model
        X = self.initialize_X(X) #reformats X
        
        
        loss_list = np.zeros(iter,) # We will store the values of the loss function as gradient descent advances
        
        for i in range(iter):
            weights = self.gradient_descent_step(X, y, weights, alpha)
            
            loss_list[i] = self.loss_function(X,y,weights)
            
            if loss_list[i] <= epsilon:
                break
            
        self.weights = weights
        
        return loss_list
    
    def predict(self,X):
        '''
        Predicts labels y given an input matrix X
        Input: 
        X- matrix of dimensions N x D
        
        Output:
        y_pred - vector of labels (dimensions N x 1)
        '''
        #1) Reformat the matrix X
        #YOUR CODE HEREX)
        
        #2) Estimate Xweights
        #YOUR CODE HERE
        
        #3) Use slide 24 from the slide deck to assign the labels y
        #YOUR CODE HERE
        
        return y_pred.astype(int)
        

We are now ready to test your implementation of Logistic Regression. Go through the different steps below and understand what exactly is being done. 

In [ ]:
#First we split the data into two sets: training and testing (no validation set in this lab)
X_tr, X_te, y_tr, y_te = train_test_split(X,y,test_size=0.1)

# Next we standardize the training set
X_tr, mean, std = standardize(X_tr)

# The test input features are standardized using the mean and std computed on the training set 
X_te, _, _ = standardize(X_te, mean, std)

#We initialize the logistic regression class
logistic = LogisticRegression() 

y_tr = y_tr.reshape((len(X_tr),1))
y_te = y_te.reshape((len(X_te),1))
#We fit the model using a learning rate of 0.01 and 500 iterations
values = logistic.fit(X_tr,y_tr, 0.01, 500)      

In [ ]:
plot_loss('Evolution of the loss function, alpha = 0.1, iter = 500', values)

Now, we estimate labels for the training and the testing dataset. Then we will assess the performance using the [F1-score](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.f1_score.html)

In [ ]:
y_train_pred = logistic.predict(X_tr)
y_test_pred = logistic.predict(X_te)

print(f'Performance in the training set:{f1_score(y_tr, y_train_pred)}\n')
print(f'Performance in the test set:{f1_score(y_te, y_test_pred)}\n')

How does your model perform? Are you satisfied? Comment

Your answer here:

### Task 2.2: Varying the learning rate and the number of iterations
Run multiple times the fit function, using different values of the learning rate (0.001 and 0.1) and the iterations (500 and 1000). Comment on your results.

In [ ]:
#CODE HERE

Comment on the obtained curves. How does the behavior of the loss changes?

Your answer here: 

## Optional Exercise: Changing the Properties of the Data (Bonus point)
Play around with the make_classification function by varying the number of redundant, repeated and informative features. For each new dataset you generate, train the logistic regression classifier. Comment on the results you obtained. What happens when there are too many redundant and/or repeated features? Too many random ones? How does the number of informative features affect the quality of the classifier?